<a href="https://colab.research.google.com/github/pcsilcan/aed/blob/master/week11/11_SimpleBST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
%%writefile simplebst.h

#ifndef __SIMPLEBST_H__
#define __SIMPLEBST_H__

#include <functional>

class BST {
    struct Node {
        float element;
        Node* left;
        Node* right;

        Node(float element)
            : element(element), left(nullptr), right(nullptr) {}
    };

    Node* root;
    int len;

public:
    BST() : root(nullptr), len(0) {}
    ~BST() {
        clear(root);
    }

    void add(float elem) {
        add(root, elem);
    }

    bool find(float elem) {
        find(root, elem);
    }

    void preorder(std::function<void(float)> proc) {
        preorder(root, proc);
    }

    void postorder(std::function<void(float)> proc) {
        postorder(root, proc);
    }

    void inorder(std::function<void(float)> proc) {
        inorder(root, proc);
    }

private:
    void clear(Node* node) {
        if (node != nullptr) {
            clear(node->left);
            clear(node->right);
            delete node;
        }
    }

    void add(Node*& node, float elem) {
        if (node == nullptr) {
            node = new Node(elem);
        } else if (elem < node->element) {
            add(node->left, elem);
        } else {
            add(node->right, elem);
        }
    }

    bool find(Node* node, float elem) {
        if (node != nullptr) {
            if (elem == node->element) {
                return true;
            } else if (elem < node->element) {
                return find(node->left, elem);
            } else {
                return find(node->right, elem);
            }
        } else {
            return false;
        }
    }

    void preorder(Node* node, std::function<void(float)> proc) {
        if (node != nullptr) {
            preorder(node->left, proc);
            preorder(node->right, proc);
            proc(node->element);
        }
    }

    void postorder(Node* node, std::function<void(float)> proc) {
        if (node != nullptr) {
            proc(node->element);
            postorder(node->left, proc);
            postorder(node->right, proc);
        }
    }

    void inorder(Node* node, std::function<void(float)> proc) {
        if (node != nullptr) {
            inorder(node->left, proc);
            proc(node->element);
            inorder(node->right, proc);
        }
    }
};

#endif

Overwriting simplebst.h


In [9]:
%%writefile bstTest.cpp

#include <iostream>
#include "simplebst.h"

using namespace std;

int main() {
    BST* bst = new BST();
    bst->add(1);
    bst->add(12);
    bst->add(34);
    bst->add(61);
    bst->add(85);
    bst->add(52);
    bst->add(92);
    bst->add(2);
    bst->add(6);
    bst->add(10);
    bst->add(45);
    bst->add(78);
    bst->add(96);
    bst->add(33);
    bst->add(10);
    bst->add(71);
    bst->add(8);
    bst->add(13);

    auto print = [](float x) { cout << x << " "; };
    bst->preorder(print); cout << endl;
    bst->postorder(print); cout << endl;
    bst->inorder(print); cout << endl;

    cout << bst->find(52) << endl;
    cout << bst->find(100) << endl;

    delete bst;
    return 0;
}

Writing bstTest.cpp


In [14]:
%%script bash

g++ -std=c++17 *.cpp && ./a.out

8 10 10 6 2 13 33 45 52 71 78 96 92 85 61 34 12 1 
1 12 2 6 10 8 10 34 33 13 61 52 45 85 78 71 92 96 
1 2 6 8 10 10 12 13 33 34 45 52 61 71 78 85 92 96 
1
0
